In [1]:
!git clone https://github.com/jma127/pyltr.git

Cloning into 'pyltr'...
remote: Enumerating objects: 448, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 448 (delta 8), reused 12 (delta 3), pack-reused 419
Receiving objects: 100% (448/448), 84.51 KiB | 14.08 MiB/s, done.
Resolving deltas: 100% (260/260), done.


In [2]:
!pip install deap
#!pip install --upgrade scikit-learn

     |████████████████████████████████| 163kB 27.2MB/s 


In [3]:
import operator
import math
import random
import numpy as np
import datetime
from math import log, inf
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp
import subprocess

import pyltr.pyltr as pyltr

import warnings
warnings.filterwarnings('ignore')

In [4]:
with open('/content/drive/My Drive/Colab Notebooks/LETOR/MQ2008/Fold1/train.txt') as trainfile:
    TrainX, Trainy, Trainqids, _ = pyltr.data.letor.read_dataset(trainfile)

with open('/content/drive/My Drive/Colab Notebooks/LETOR/MQ2008/Fold1/test.txt') as testfile:
    TestX, Testy, Testqids, _ = pyltr.data.letor.read_dataset(testfile)

#with open('/content/drive/My Drive/Colab Notebooks/LETOR/MQ2007/Fold1/vali.txt') as valifile:
#    ValiX, Valiy, Valiqids, _ = pyltr.data.letor.read_dataset(valifile)

In [5]:
def print_rank_documents(pop, dataset_x, dataset_y, dataset_query):
  individual = tools.selBest(pop, 1)[0]
  rank_function = toolbox.compile(expr=individual)
  all_documents = []
  true_label = {}
  true_label['0'] = "Not relevant"
  true_label['1'] = "Indiferente"
  true_label['2'] = "Relevant"  
  
  with open("/content/drive/My Drive/Colab Notebooks/LETOR/result.txt", "a") as fr:
    last_query_id = dataset_query[0]
    for row, query_id in enumerate(dataset_query):
      if (last_query_id != query_id):
        fr.write("\n#QueryId: " + str(last_query_id) + " ---------------------------------------------------------------------------\n")
        print("#QueryId: " + str(last_query_id) + " ---------------------------------------------------------------------------")
        #Sort documents by score
        all_documents = sorted(all_documents, key=lambda x: x[0], reverse=True)

        for doc in all_documents:
          fr.write("    > " + true_label[str(int(doc[1]))] + " - #DocId: " + str(doc[2]) + ", Score: " + str(round(doc[0], 2)) + "\n")
          print("    > " + true_label[str(int(doc[1]))] + " - #DocId: " + str(doc[2]) + ", Score: " + str(round(doc[0], 2)) )
        
        all_documents = []
        last_query_id = query_id
      
      features = list(dataset_x[row])
      score = rank_function(*features)
      all_documents.append([score, dataset_y[row], row])

def get_best_of_test(pop, dataset_x):
  population = tools.selBest(pop, 10)
  all_ind = []
  for individual in population:
    rank_function = toolbox.compile(expr=individual)
    
    scores = [rank_function(*features) for features in dataset_x]

    with open("rank", "w") as f:
      for score in scores:
          f.write("%s\n" % score)

    process = subprocess.Popen(['/usr/bin/perl', 'Eval-Score-4.0-train.pl', 'MQ2008/test.txt', 'rank', 'info', '0'], stdout=subprocess.PIPE)
    metrics = process.stdout.readline().split(b';')
    ndcg = np.float64(metrics[1])
    map =  np.float64(metrics[0])
    all_ind.append([ndcg, individual])
  
  all_ind = sorted(all_ind, key=lambda x: x[0], reverse=True)
  return all_ind[0][1]

In [6]:
def div_(left, right):
    return 1.0 if right == 0.0 else (left / right)

def log2_(value):
  return math.log2(value) if value > 1.0 else 0.0

def calc_metrics(individual, dataset_x):
  rank_function = toolbox.compile(expr=individual)
  
  scores = [rank_function(*features) for features in dataset_x]

  with open("rank", "w") as f:
    for score in scores:
        f.write("%s\n" % score)

  process = subprocess.Popen(['/usr/bin/perl', 'Eval-Score-4.0-train.pl', 'MQ2008/train.txt', 'rank', 'info', '0'], stdout=subprocess.PIPE)
  metrics = process.stdout.readline().split(b';')
  ndcg = np.float64(metrics[1])
  map =  np.float64(metrics[0])

  return ndcg, map

def calc_metrics_test(pop, dataset_x):
  individuals = tools.selBest(pop, 10)
  ind = 1
  for individual in individuals:
    print("Individuo: " + str(ind))
    
    print("Função de ranqueamento: " + str(individual))
    rank_function = toolbox.compile(expr=individual)
    scores = [rank_function(*features) for features in dataset_x]

    with open("rank", "w") as f:
      for score in scores:
          f.write("%s\n" % score)
    
    subprocess.call(['/usr/bin/perl', 'Eval-Score-4.0.pl', 'MQ2008/test.txt', 'rank', 'info', '0'])
    with open("info", "r") as fi:
      with open("/content/drive/My Drive/Colab Notebooks/LETOR/result.txt", "a") as fr:
        fr.write("\nIndividuo: " + str(ind)+"\n")
        fr.write("Função de ranqueamento: " + str(individual)+"\n\n")
        fr.write(fi.read() + "\n\n\n")
        print(fi.read())
    ind += 1

def eval_fitness(individual, dataset_x):
    ndcg, map = calc_metrics(individual, dataset_x)
    return ndcg,

pset = gp.PrimitiveSet("MAIN", 46)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(div_, 2) 
#pset.addPrimitive(log2_, 1)
pset.addTerminal(0.0, 'C0_0')
pset.addTerminal(0.1, 'C0_1')
pset.addTerminal(0.2, 'C0_2')
pset.addTerminal(0.3, 'C0_3')
pset.addTerminal(0.4, 'C0_4')
pset.addTerminal(0.5, 'C0_5')
pset.addTerminal(0.6, 'C0_6')
pset.addTerminal(0.7, 'C0_7')
pset.addTerminal(0.8, 'C0_8')
pset.addTerminal(0.9, 'C0_9')
pset.addTerminal(1.0, 'C1_0')
pset.addTerminal(1.1, 'C1_1')
pset.addTerminal(1.2, 'C1_2')
pset.addTerminal(1.3, 'C1_3')
pset.addTerminal(1.4, 'C1_4')
pset.addTerminal(1.5, 'C1_5')
pset.addTerminal(1.6, 'C1_6')
pset.addTerminal(1.7, 'C1_7')
pset.addTerminal(1.8, 'C1_8')
pset.addTerminal(1.9, 'C1_9')
pset.addTerminal(2.0, 'C2_0')

pset.renameArguments(ARG0='tf_body')
pset.renameArguments(ARG1='tf_anchor')
pset.renameArguments(ARG2='tf_title')
pset.renameArguments(ARG3='tf_url')
pset.renameArguments(ARG4='tf_doc')

pset.renameArguments(ARG5='idf_body')
pset.renameArguments(ARG6='idf_anchor')
pset.renameArguments(ARG7='idf_title')
pset.renameArguments(ARG8='idf_url')
pset.renameArguments(ARG9='idf_doc')

pset.renameArguments(ARG10='tf_idf_body')
pset.renameArguments(ARG11='tf_idf_anchor')
pset.renameArguments(ARG12='tf_idf_title')
pset.renameArguments(ARG13='tf_idf_url')
pset.renameArguments(ARG14='tf_idf_doc')

pset.renameArguments(ARG15='doc_len_body')
pset.renameArguments(ARG16='doc_len_anchor')
pset.renameArguments(ARG17='doc_len_title')
pset.renameArguments(ARG18='doc_len_url')
pset.renameArguments(ARG19='doc_len_doc')

pset.renameArguments(ARG20='bm25_body')
pset.renameArguments(ARG21='bm25_anchor')
pset.renameArguments(ARG22='bm25_title')
pset.renameArguments(ARG23='bm25_url')
pset.renameArguments(ARG24='bm25_doc')

pset.renameArguments(ARG25='lmir_abs_body')
pset.renameArguments(ARG26='lmir_abs_anchor')
pset.renameArguments(ARG27='lmir_abs_title')
pset.renameArguments(ARG28='lmir_abs_url')
pset.renameArguments(ARG29='lmir_abs_doc')

pset.renameArguments(ARG30='lmir_dir_body')
pset.renameArguments(ARG31='lmir_dir_anchor')
pset.renameArguments(ARG32='lmir_dir_title')
pset.renameArguments(ARG33='lmir_dir_url')
pset.renameArguments(ARG34='lmir_dir_doc')

pset.renameArguments(ARG35='lmir_jm_body')
pset.renameArguments(ARG36='lmir_jm_anchor')
pset.renameArguments(ARG37='lmir_jm_title')
pset.renameArguments(ARG38='lmir_jm_url')
pset.renameArguments(ARG39='lmir_jm_doc')

pset.renameArguments(ARG40='pagerank')
pset.renameArguments(ARG41='num_inlink')
pset.renameArguments(ARG42='num_outlink')
pset.renameArguments(ARG43='num_slash_url')
pset.renameArguments(ARG44='len_url')
pset.renameArguments(ARG45='num_child_page')

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=2, max_=10)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

toolbox.register("evaluate", eval_fitness, dataset_x=TrainX)
toolbox.register("select", tools.selTournament, tournsize=4)
toolbox.register("mate", gp.cxOnePointLeafBiased, termpb=0.1)
toolbox.register("expr_mut", gp.genHalfAndHalf, min_=2, max_=10)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)
toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter('height'), max_value=30))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter('height'), max_value=30))

def train():
    np.random.seed(245)

    pop = toolbox.population(n=500)

    #Elitism
    hof = tools.HallOfFame(1)
    
    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("avg", np.mean)
    mstats.register("std", np.std)
    mstats.register("min", np.min)
    mstats.register("max", np.max)

    pop, log = algorithms.eaMuCommaLambda(pop, #Population                                  
                                   toolbox,
                                   mu=50, #The number of individuals to select for the next generation
                                   lambda_=450, #The number of children to produce at each generation
                                   cxpb=0.85, #crossover
                                   mutpb=0.08, #mutation
                                   ngen=35, #Num. generation
                                   stats=mstats, #Statistics
                                   halloffame=hof, #Elitism
                                   verbose=True)
    
    return pop, log, hof

#Training & Evaluate

In [ ]:
NUM_ITERATION = 10
list_best_average = np.zeros(NUM_ITERATION, dtype=np.float64)
all_best_ind = []
for i in range(NUM_ITERATION):
  with open("/content/drive/My Drive/Colab Notebooks/LETOR/result.txt", "a") as fr:
    fr.write("\n\n\n>>> Num. Iteration: " + str(i+1)+"\n")
    fr.write(str(datetime.datetime.now()) + "\n\n" )
  print(">>> Num. Iteration: " + str(i+1))
  print(">>> Init Training\n")

  pop, log, hof = train()
  print(">>> Init Test")
  calc_metrics_test(pop, TestX)
  all_best_ind.append(get_best_of_test(pop, TestX))
with open("/content/drive/My Drive/Colab Notebooks/LETOR/result.txt", "a") as fr:
  fr.write("\n" + str(datetime.datetime.now()))
print_rank_documents(pop, TestX, Testy, Testqids)

>>> Num. Iteration: 1
>>> Init Training

   	      	                         fitness                         	                      size                     
   	      	---------------------------------------------------------	-----------------------------------------------
gen	nevals	avg     	gen	max   	min   	nevals	std      	avg    	gen	max 	min	nevals	std    
0  	500   	0.298589	0  	0.4704	0.1495	500   	0.0553987	213.992	0  	2047	7  	500   	486.986
1  	423   	0.33972 	1  	0.4563	0.2827	423   	0.0468108	289.36 	1  	4087	5  	423   	745.838
2  	418   	0.372196	2  	0.4777	0.2992	418   	0.0480248	23.16  	2  	483 	3  	418   	67.0138
3  	415   	0.406246	3  	0.4777	0.3209	415   	0.0459953	14.08  	3  	59  	3  	415   	11.301 
4  	421   	0.436886	4  	0.4798	0.3674	421   	0.0323515	97.08  	4  	2069	3  	421   	402.62 
5  	425   	0.462348	5  	0.4835	0.3543	425   	0.0260564	54.56  	5  	2057	5  	425   	286.125
6  	418   	0.475148	6  	0.4866	0.4432	418   	0.00998752	14.76  	6  	29  	5  	418   	5.81

In [ ]:
individuals = tools.selBest(pop, 10)
ind = 0
for individual in individuals:
  print("Individuo: " + str(ind))
  
  print("Função de ranqueamento: " + str(individual))
  rank_function = toolbox.compile(expr=individual)
  scores = [rank_function(*features) for features in TestX]

  ind += 1
  with open("rank_test" + str(ind), "w") as f:
    for score in scores:
        f.write("%s\n" % score)

Individuo: 0
Função de ranqueamento: add(mul(C1_3, lmir_jm_url), mul(mul(mul(mul(C0_4, lmir_abs_url), mul(div_(mul(C1_3, mul(sub(mul(mul(add(sub(div_(div_(doc_len_url, lmir_abs_url), div_(bm25_title, C0_4)), div_(div_(lmir_jm_anchor, C1_3), add(C0_0, lmir_jm_anchor))), div_(div_(mul(C0_3, tf_anchor), add(C1_2, C0_2)), div_(sub(C1_0, tf_idf_title), div_(C0_7, lmir_abs_url)))), sub(add(sub(add(C1_1, C1_3), sub(lmir_jm_anchor, C1_0)), add(mul(bm25_url, lmir_jm_url), add(doc_len_title, idf_url))), mul(mul(sub(C0_4, doc_len_title), div_(num_inlink, C1_0)), sub(mul(bm25_title, C1_3), add(bm25_doc, C1_0))))), div_(mul(add(add(div_(C0_8, C0_3), add(lmir_dir_doc, tf_idf_anchor)), mul(sub(bm25_anchor, tf_doc), sub(C0_1, tf_idf_anchor))), div_(mul(add(lmir_jm_url, C0_3), mul(num_inlink, lmir_dir_url)), div_(add(lmir_jm_title, doc_len_body), mul(tf_url, lmir_dir_url)))), mul(mul(C0_4, lmir_abs_url), mul(mul(C1_5, lmir_abs_url), C0_4)))), mul(add(mul(mul(sub(add(doc_len_url, idf_doc), sub(doc_len_b

In [ ]:
def convert(input_filename, out_data_filename, out_query_filename, out_query_filename2):
	input = open(input_filename,"r")
	output_feature = open(out_data_filename,"w")
	output_query = open(out_query_filename,"w")
	output_query2 = open(out_query_filename2,"w")
	cur_cnt = 0
	cur_doc_cnt = 0
	last_qid = -1
	while True:
		line = input.readline()
		if not line:
			break
		tokens = line.split(' ')
		tokens[-1] = tokens[-1].strip()
		label = tokens[0]
		qid = int(tokens[1].split(':')[1])
		if qid != last_qid:
			if cur_doc_cnt > 0:
				output_query.write(str(cur_doc_cnt) + '\n')
				output_query2.write(str(cur_doc_cnt) + '\n')
				cur_cnt += 1
			cur_doc_cnt = 0
			last_qid = qid
		cur_doc_cnt += 1
		output_feature.write(label+' ')
		output_feature.write(' '.join(tokens[2:]) + '\n')
	output_query.write(str(cur_doc_cnt) + '\n')
	output_query2.write(str(cur_doc_cnt) + '\n')
	
	input.close()
	output_query.close()
	output_feature.close()
	output_query2.close()

convert("train.txt","msltr.train","msltr.train.query","msltr.train.group")
convert("test.txt","msltr.test","msltr.test.query","msltr.test.group")